# Developing REST APIs with Amazon API Gateway

## Overview and objectives
In this project, I will create a REST application programming interface (API) by using Amazon API Gateway.

After reading this document, you also should be able to:

- Create simple mock endpoints for REST APIs and use them in your website.
- Enable Cross-Origin Resource Sharing (CORS)

## Scenario
In the previous lab, I took on the role of Sofía to build a web application for the café. As part of this process, I created an Amazon DynamoDB table that was named FoodProducts, where I stored information about café menu items.

I then loaded data that was formatted in JavaScript Object Notation (JSON) into the database table. The table structure looked similar to the following table (one line item of table data is shown as an example):

| product_name | description | price_in_cents |product_id | tags3 | special |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|  apple pie slice   |  A delicious slice of Frank's homemade pie.   |  595   |  a444   |  [ { "S" : "pie slice" }, { "S" : "on offer" } ]   |  1   |

In the previous lab I also configured code that used the AWS SDK for Python (Boto3) to:

- Scan a DynamoDB table to retrieve product details.
- Return a single item by product name using get-item as a proof of concept
- Create a Global Secondary Index (GSI) called special_GSI that you could use to filter out menu items that are on offer and not out of stock.

In this project, I will continue the role of Sofía. I will use Amazon API Gateway to configure mock data endpoints. There are three that I will create:

- [GET] /products (which will eventually invoke a DynamoDB table scan)
- [GET] /products/on_offer (which will eventually invoke a DynamoDB index scan and filter)
- [POST] /create_report (which will eventually trigger a batch process that will send out a report)

Then in the [project that follows this one](), I will replace the mock endpoints with real endpoints, so that the web application can connect to the DynamoDB backend.

When you start the AWS simulation, the following resources are pre-created in the account.

![start-arch](images/start-arch.png)

However, by the end of this project, I have created the following architecture:

![end-arch](images/end-arch.png)



# Task 1: Preparing the development environment

In this first task, I will configure your AWS Cloud9 environment so that you can create the REST API.

From the AWS Management Console, connect to the AWS Cloud9 IDE named Cloud9 Instance.
 
![cloud9-instance](images/cloud9-instance.png)

Download and extract the files that you will need for this lab.

In the same terminal, run the following command:

In [ ]:
wget https://aws-tc-largeobjects.s3.us-west-2.amazonaws.com/CUR-TF-200-ACCDEV-2-91558/04-lab-api/code.zip -P /home/ec2-user/environment

Notice that a code.zip file was downloaded to the AWS Cloud9 instance. The file is listed in the Environment window.

![restcodezip](images/restcodezip.png)

Extract the file:

In [ ]:
unzip code.zip

![restunzip](images/restunzip.png)

Run the script that upgrades the versions of Python and the AWS CLI installed in your IDE environment,  and also creates the cafe website in your AWS account.

In [ ]:
chmod +x resources/setup.sh && resources/setup.sh

The script will prompt you for the IP address by which your computer is known to the internet.

![rest-chmod](images/rest-chmod.png)

Use www.whatismyip.com to discover this address and then paste the IPv4 address into the command prompt and finish running the script.

### Verify the version of AWS CLI installed.

In the AWS Cloud9 Bash terminal (at the bottom of the IDE), run the following command:

In [ ]:
aws --version

In [ ]:
aws-cli/2.22.34 Python/3.12.6 Linux/5.10.230.223.885.amzn2.x86_64.exe

The output should indicate that version 2 is installed.

### Verify that the SDK for Python is installed.

Run the following command:

In [ ]:
pip show boto3

![rest-boto](images/rest-boto.png)

Note: If you see a message about not using the latest version of pip, ignore the message.

### Verify that the cafe website can be loaded in a browser tab.

Load the website in a browser tab.

In a browser tab, open the Amazon S3 console.

Choose your bucket name, and then choose Objects.

If the files that the script just uploaded do not display, choose the refresh icon to view them.

Choose the index.html file.

Copy the Object URL. It will be in the following format.

In [ ]:
https://<bucket-name>.s3.amazon.com/index.html

![rest-index](images/rest-index.png)

Verify that the website displays by pasting the full URL into your browser.

![rest-cafe](images/rest-cafe.png)

### Note particular website details.

Notice in the Browse Pastries section that there are two buttons.

The "on offer" view displays by default and it shows six menu items.

Select the "view all" view. Notice that many more menu items display.

![cafe-onoffer-viewall](images/cafe-onoffer-viewall.png)

Optionally, expose the developer console view in your browser.

For example, if you are using Chrome, choose View > Developer > View JavaScript Console
If you are using Firefox, choose Tools > Web Developer > Web Console.
Analysis: These menu details that display on the website are currently being read out of the all_products_on_offer.json and all_products.json files that are hosted in your S3 bucket. You also have a copy of these same files in your Cloud9 IDE, in the resources/website directory.

If you are looking at the browser developer console view details, you will also see a log entry written by the main.js file, indicating that hardcoded data is being used.  Later in the lab, you will see that the number of menu items that display changes and that the messages in the console change as well.

Leave the website open in this browser tab, you will return to it towards the end of the lab.

## Task 2: Creating the first API endpoint (GET)

In this task, I will create a REST API called ProductsApi. I will also create the first of three resources for the API.

The first API resource will be called products. It will make a GET request so that the website can retrieve all rows from the FoodProducts DynamoDB database table. You will then deploy it in an API Gateway stage that's named prod. When a user visits the website, it will make an AJAX request and return a list of café menu items from API gateway (it will return mock data for now).

To complete all these tasks, I will use the SDK for Python.

In the AWS Cloud9 navigation pane, expand the python_3 directory and open the file named create_products_api.py.
 

On line 3, replace the  (fill me in) with the correct value that will create an API Gateway client.

 Tip: Consult the [SDK for Python documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/apigateway.html) as needed.

![create-products-fmi](images/create-products-fmi.png)

### Take a moment to analyze the first part of what this code will do when you run it:

Lines 5-24 create a REST API that's named ProductsApi, and a resource that's named products.

In [ ]:
response = client.create_rest_api(
    name='ProductsApi',
    description='API to get all the food products.',
    minimumCompressionSize=123,
    endpointConfiguration={
        'types': [
            'REGIONAL',
        ]
    }
)
api_id = response["id"]

resources = client.get_resources(restApiId=api_id)
root_id = [resource for resource in resources["items"] if resource["path"] == "/"][0]["id"]

products = client.create_resource(
    restApiId=api_id,
    parentId=root_id,
    pathPart='products'
)

Lines 28-33 create a method request of type GET in the products resource.


In [ ]:
product_method = client.put_method(
    restApiId=api_id,
    resourceId=products_resource_id,
    httpMethod='GET',
    authorizationType='NONE'
)

You will analyze what the additional lines of code accomplish later in this task.

Run the code.

Save the change to the file.

Then, in the Bash terminal, go to the directory that contains the Python code file, and run the code.

In [ ]:
cd python_3
python create_products_api.py

![create-products-apipy](images/create-products-apipy.png)

Return to the AWS Management Console browser tab, and open the API Gateway console.
 

Open the ProductsApi that you just created by choosing the link.
 

Choose the GET method that you defined.

You should see the details of the GET method execution in a graphical format.

![endpoint1](images/endpoint1.png)

Take a moment to study the data flow in the GET method that you defined.  

Tip: If you have a screen that is large enough, arrange the browser tabs so that you have the AWS Cloud9 IDE open next to this browser tab. This way, you can see the code and what it produced side-by-side.

On the left is the Client.
- Lines 28-33 - When you run the Test, the Method Request is sent to the URL in the Amazon Resource Name (ARN) detail. The request doesn't require any authorization to invoke it.
- Lines 50-58 - The Integration Request of type MOCK is invoked, and the mock endpoint receives the data.
- Lines 35-48 - The mock endpoint invokes the Integration Response, which invokes the Method Response.
- Lines 61-92 - The Method Response returns the REST API response back to the Client that the request originated from.

#### Analysis: To make it easier during this initial API development phase, you will use mock data. When you test the API call, it will not actually connect to the database. Instead, it will return the data that's hardcoded in the responseTemplate part of the code (lines 67-91).  

This approach reduces the scope of potential errors during testing. You can stay focused in this lab on ensuring that the REST API logic is well defined.    	

However, the structure of this mock data intentionally matches the data structure that will appear in the next lab when Lambda will be interacting with the database table.

The key values will be mapped to the attributes that are defined in the DynamoDB table (which you created in the previous lab).

Note: attributes in DynamoDB are not primatives. Instead, they are wrapper objects (as shown in the example code below). This is why there is a slight difference between the key names in the JSON and the attribute names in DynamoDB.



In [ ]:
product_name: {
 "S": "vanilla cupcake"
}

In the API Gateway console, choose the  TEST link, then scroll to the bottom and choose the Test button.

In the panel on the right, you should see the following response body, response headers, and log information.

In [ ]:
[
  {
    "product_name_str": "apple pie slice",
    "product_id_str": "a444",
    "price_in_cents_int": 595,
    "description_str": "amazing taste",
    "tag_str_arr": [
      "pie slice",
      "on offer"
    ],
    "special_int": 1
  },
  {
    "product_name_str": "chocolate cake slice",
    "product_id_str": "a445",
    "price_in_cents_int": 595,
    "description_str": "chocolate heaven",
    "tag_str_arr": [
      "cake slice",
      "on offer"
    ]
  },
  {
    "product_name_str": "chocolate cake",
    "product_id_str": "a446",
    "price_in_cents_int": 4095,
    "description_str": "chocolate heaven",
    "tag_str_arr": [
      "whole cake",
      "on offer"
    ]
  }
]

![get-method-test1](images/get-method-test1.png)

Congratulations! You have now successfully created and tested a REST API with a resource that makes a GET request.

## Task 3: Creating the second API endpoint (GET)

In this task, I will define another API endpoint of type GET. This endpoint will eventually support calls to/products/on_offer from the cafe website and it will return in stock items.

 
In the AWS Cloud9 navigation pane, expand the python_3 directory and open the file named create_on_offer_api.py.

Replace <FMI_1> and <FMI_2> with the correct values so that this code file will add another resource to the API that you defined in the previous task.

In a browser tab, go to the API Gateway console and choose the ProductsApi API that you created a moment ago.

In the panel on the left, choose Resources.

Choose GET under products

In the breadcrumb navigation at the top of the screen (above the Actions menu), you can see APIs >  ProductsAPI followed by an id in parenthesis.  

This is the api_id.
On the same line,  you will see /products, followed by another id in parenthesis.  

This is the resource  parent_id

![onofferfmi1fmi2](images/onofferfmi1fmi2.png)


Observe the rest of the code.

The code looks very similar to the create_products_api code, because it is also creating a GET resource with a mock data endpoint.

Notice that the hardcoded data response is formatted as shown below. It returns a single menu item, which will be sufficient since this is mock data.

In [ ]:
[
  {
    "product_name_str": "apple pie slice",
    "product_id_str": "a444",
    "price_in_cents_int": 595,
    "description_str": "amazing taste",
    "tag_str_arr": [
      "pie slice",
      "on offer"
    ],
    "special_int": 1
  }
]

Create the API resource.

Save the change to the file.

Then in the Bash terminal, verify that the current directory is python_3 and run the code.

In [ ]:
python create_on_offer_api.py

Observe the results.

Return to the AWS Management Console browser tab, and open the API Gateway console.

Choose the APIs link in the breadcrumb navigation above, then on the left, open the ProductsApi by choosing the link.

Notice that there is now a nested resource called /on_offer under the /products resource.

![task3](images/task3.png)

Test the /on_offer resource.

Use the  Test link, the same way you tested the first resource in the previous task.

You should receive a 200 HTML status code response.

In [ ]:
[
  {
    "product_name_str": "apple pie slice",
    "product_id_str": "a444",
    "price_in_cents_int": 595,
    "description_str": "amazing taste",
    "tag_str_arr": [
      "pie slice",
      "on offer"
    ],
    "special_int": 1
  }
]

![onoffer-test](images/onoffer-test.png)